In [1]:
import tensorflow as tf
import numpy as np
from StochasticDropoutNet_unrolled import StochasticDropoutNet
from util import xor_data, extract_update_dict
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# model = StochasticDropoutNet(min_dropout_rate = 0.001, max_dropout_rate = 0.001, num_weight_train_steps = 64)
model = StochasticDropoutNet(min_init_dropout_rate = 0.5, 
                             max_init_dropout_rate = 0.5,
                             train_batch_size = 128,
                             valid_batch_size = 256,
                             unroll_steps = 10)

In [ ]:
inputs_train, targets_train = mnist.train.next_batch(55000)
inputs_train = inputs_train.reshape((-1, 28, 28, 1))
inputs_valid, targets_valid = mnist.validation.next_batch(5000)
inputs_valid = inputs_valid.reshape((-1, 28, 28, 1))

model.train_unroll(inputs_train, targets_train, inputs_valid, targets_valid, num_epochs = 40)
model.train_fine_tune(inputs_train, targets_train, inputs_valid, targets_valid, num_epochs = 20)

Epoch 0, weight train batch 0, step 0, loss = 3546.24560547, accuracy = 0.140625
Epoch 0, weight train batch 1, step 1, loss = 19890.2421875, accuracy = 0.140625
Epoch 0, weight train batch 2, step 2, loss = 64275.84375, accuracy = 0.1328125
Epoch 0, weight train batch 3, step 3, loss = 62225.9101562, accuracy = 0.1015625
Epoch 0, struct train step 1, loss = 55157.390625, accuracy = 0.1015625
Epoch 0, weight train batch 4, step 0, loss = 28906.4726562, accuracy = 0.171875
Epoch 0, weight train batch 5, step 1, loss = 9153.38476562, accuracy = 0.1015625
Epoch 0, weight train batch 6, step 2, loss = 2145.07348633, accuracy = 0.1015625
Epoch 0, weight train batch 7, step 3, loss = 471.600708008, accuracy = 0.1015625
Epoch 0, struct train step 2, loss = 482.58392334, accuracy = 0.1015625
Epoch 0, weight train batch 8, step 0, loss = 277.859802246, accuracy = 0.09375
Epoch 0, weight train batch 9, step 1, loss = 178.258178711, accuracy = 0.0859375
Epoch 0, weight train batch 10, step 2, los

In [4]:
import csv
with open('./log/results.csv', 'a+') as f:
    writer = csv.writer(f)
    writer.writerow(model.train_losses)
    writer.writerow(model.valid_losses)

In [6]:
feed_dict = {model.inputs: inputs_train[0 : 32, ...],
             model.targets: targets_train[0 : 32, ...]}
print(model.pred)
print(model.sess.run(model.pred, feed_dict = feed_dict))
print(model.sess.run(model.targets, feed_dict = feed_dict))
print(model.struct_train_op)

Tensor("Cast_1:0", shape=(?,), dtype=int32)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0 1]
[<tf.Operation 'Adam_3' type=NoOp>, <tf.Tensor 'Assign_12:0' shape=(1, 1, 128) dtype=float32_ref>, <tf.Tensor 'Assign_13:0' shape=(1, 1, 128) dtype=float32_ref>, <tf.Tensor 'Assign_14:0' shape=(1, 1, 128) dtype=float32_ref>]


In [3]:
feed_dict = dict(zip([model.inputs] + model.inputs_adapt + [model.targets] + model.targets_adapt,
                 [np.ones((1000, 1, 1, 1))] * (1 + model.unroll_steps)
                 + [np.ones((1000, 1, 1, 1))] * (1 + model.unroll_steps)))

print(model.sess.run(tf.gradients(model.unrolled_loss, model.struct_param), feed_dict = feed_dict))
print(model.sess.run(tf.gradients(model.loss, model.struct_param), feed_dict = feed_dict))
print(model.sess.run([tf.gradients(model.unrolled_loss, model.weights_adapt[9]),
                      [tf.gradients(w, model.struct_param) for w in model.weights_adapt[9]]], feed_dict = feed_dict))

[array([[[ 0.42292321]]], dtype=float32)]
[array([[[ 0.55817735]]], dtype=float32)]
[[array([[[[-0.00154307]]]], dtype=float32), array([[[[-0.66660249]]]], dtype=float32)], [[array([[[  1.48260347e-06]]], dtype=float32)], [array([[[ 0.0001383]]], dtype=float32)]]]
